# Gorilla Data Engineer Assessment
## Setup

In [1]:
import platform

print(platform.python_version())

3.11.4


In [17]:
!pip freeze | grep -f requirements.txt

jupyter==1.0.0
jupyter-console==6.6.3
jupyter-events==0.9.0
jupyter-lsp==2.2.1
jupyter_client==8.6.0
jupyter_core==5.5.0
jupyter_server==2.11.1
jupyter_server_terminals==0.4.4
jupyterlab==4.0.9
jupyterlab-widgets==3.0.9
jupyterlab_pygments==0.3.0
jupyterlab_server==2.25.2
matplotlib==3.8.2
matplotlib-inline==0.1.6
numpy==1.26.2
openpyxl==3.1.2
pandas==2.1.3
seaborn==0.13.0


## Load data

In [102]:
import pandas as pd
import numpy as np
from datetime import datetime

df_meter_list = pd.read_excel(r'data/gorilla_test_data.xlsx', sheet_name='meter_list')
df_forecast_table = pd.read_excel(r'data/gorilla_test_data.xlsx', sheet_name='forecast_table')
df_rate_table = pd.read_excel(r'data/gorilla_test_data.xlsx', sheet_name='rate_table')

In [39]:
df_meter_list.head()

,meter_id,aq_kwh,exit_zone
0,14676236,28978,EA1
1,34509937,78324,SO1
2,50264822,265667,NT1
3,88357331,484399,SE2


In [48]:
df_forecast_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   meter_id  3412 non-null   int64         
 1   date      3412 non-null   datetime64[ns]
 2   kwh       3412 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 80.1 KB


In [37]:
df_forecast_table.groupby('meter_id')['date'].agg(['min', 'max'])

,min,max
meter_id,,
14676236,2020-06-01,2022-10-01
34509937,2020-06-01,2022-10-01
50264822,2020-06-01,2022-10-01
88357331,2020-06-01,2022-10-01


In [28]:
df_rate_table.head()

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
1,2020-04-01,EA1,73200,732000.0,0.1980
2,2020-04-01,EA1,732000,NaN,0.2875
3,2020-04-01,EA2,0,73200.0,0.2970
4,2020-04-01,EA2,73200,732000.0,0.1524


## Exercise 1: Total Estimated Consumption and Total Cost

In [70]:
total_consumption = df_forecast_table.groupby('meter_id')[['kwh']].sum()
total_consumption
# total sum equals aq_kwh column in df_meter_list

,kwh
meter_id,
14676236,28978.0
34509937,78324.0
50264822,265667.0
88357331,484399.0


In [63]:
# calculate end date of period (+6m of start date)
df_rate_table = df_rate_table.rename(columns={'date': 'date_start'})
df_rate_table['date_end'] = df_rate_table['date_start'] + pd.offsets.DateOffset(months=6)
df_rate_table

,date_start,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh,date_end
0,2020-04-01,EA1,0,73200.0,0.2652,2020-10-01
1,2020-04-01,EA1,73200,732000.0,0.1980,2020-10-01
2,2020-04-01,EA1,732000,NaN,0.2875,2020-10-01
3,2020-04-01,EA2,0,73200.0,0.2970,2020-10-01
4,2020-04-01,EA2,73200,732000.0,0.1524,2020-10-01
...,...,...,...,...,...,...
1135,2024-10-01,WM2,73200,732000.0,0.4537,2025-04-01
1136,2024-10-01,WM2,732000,NaN,0.7534,2025-04-01
1137,2024-10-01,WM3,0,73200.0,0.7263,2025-04-01
1138,2024-10-01,WM3,73200,732000.0,0.6109,2025-04-01


In [68]:
df_large = df_forecast_table.merge(df_meter_list, on='meter_id').merge(df_rate_table, on=['exit_zone'])

# fill in NaN for aq_max_kwh maximum of (aq_kwh + 1) and aq_min_kwh
df_large['aq_max_kwh_clean'] = np.maximum(df_large.aq_min_kwh, df_large.aq_max_kwh.fillna(df_large.aq_kwh + 1))
df_large

,meter_id,date,kwh,aq_kwh,exit_zone,date_start,aq_min_kwh,aq_max_kwh,rate_p_per_kwh,date_end,aq_max_kwh_clean
0,14676236,2020-06-01,22.070768,28978,EA1,2020-04-01,0,73200.0,0.2652,2020-10-01,73200.0
1,14676236,2020-06-01,22.070768,28978,EA1,2020-04-01,73200,732000.0,0.1980,2020-10-01,732000.0
2,14676236,2020-06-01,22.070768,28978,EA1,2020-04-01,732000,NaN,0.2875,2020-10-01,732000.0
3,14676236,2020-06-01,22.070768,28978,EA1,2020-10-01,0,73200.0,0.2970,2021-04-01,73200.0
4,14676236,2020-06-01,22.070768,28978,EA1,2020-10-01,73200,732000.0,0.2218,2021-04-01,732000.0
...,...,...,...,...,...,...,...,...,...,...,...
102355,88357331,2022-10-01,460.535505,484399,SE2,2024-04-01,73200,732000.0,0.5739,2024-10-01,732000.0
102356,88357331,2022-10-01,460.535505,484399,SE2,2024-04-01,732000,NaN,0.5717,2024-10-01,732000.0
102357,88357331,2022-10-01,460.535505,484399,SE2,2024-10-01,0,73200.0,0.6015,2025-04-01,73200.0
102358,88357331,2022-10-01,460.535505,484399,SE2,2024-10-01,73200,732000.0,0.6428,2025-04-01,732000.0


In [74]:

# filter for correct period
df_large_period_filtered = df_large[np.logical_and(df_large.date >= df_large.date_start,
                                                   df_large.date < df_large.date_end)]
# filter for correct AQ band
df = df_large_period_filtered[np.logical_and(df_large_period_filtered.aq_kwh >= df_large_period_filtered.aq_min_kwh,
                                             df_large_period_filtered.aq_kwh < df_large_period_filtered.aq_max_kwh_clean)]
df

,meter_id,date,kwh,aq_kwh,exit_zone,date_start,aq_min_kwh,aq_max_kwh,rate_p_per_kwh,date_end,aq_max_kwh_clean
0,14676236,2020-06-01,22.070768,28978,EA1,2020-04-01,0,73200.0,0.2652,2020-10-01,73200.0
30,14676236,2020-06-02,19.170720,28978,EA1,2020-04-01,0,73200.0,0.2652,2020-10-01,73200.0
60,14676236,2020-06-03,23.555111,28978,EA1,2020-04-01,0,73200.0,0.2652,2020-10-01,73200.0
90,14676236,2020-06-04,18.220712,28978,EA1,2020-04-01,0,73200.0,0.2652,2020-10-01,73200.0
120,14676236,2020-06-05,14.196134,28978,EA1,2020-04-01,0,73200.0,0.2652,2020-10-01,73200.0
...,...,...,...,...,...,...,...,...,...,...,...
102223,88357331,2022-09-27,441.014725,484399,SE2,2022-04-01,73200,732000.0,0.3647,2022-10-01,732000.0
102253,88357331,2022-09-28,441.512055,484399,SE2,2022-04-01,73200,732000.0,0.3647,2022-10-01,732000.0
102283,88357331,2022-09-29,437.240657,484399,SE2,2022-04-01,73200,732000.0,0.3647,2022-10-01,732000.0
102313,88357331,2022-09-30,517.608354,484399,SE2,2022-04-01,73200,732000.0,0.3647,2022-10-01,732000.0


In [75]:
# calculate cost
df['cost_pound'] = df.kwh * df.rate_p_per_kwh / 100
df

/var/folders/r4/13_py2jj2ybgk5_1_92hr8kr0000gn/T/ipykernel_83184/3672623622.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cost_pound'] = df.kwh * df.rate_p_per_kwh / 100


,meter_id,date,kwh,aq_kwh,exit_zone,date_start,aq_min_kwh,aq_max_kwh,rate_p_per_kwh,date_end,aq_max_kwh_clean,cost_pound
0,14676236,2020-06-01,22.070768,28978,EA1,2020-04-01,0,73200.0,0.2652,2020-10-01,73200.0,0.058532
30,14676236,2020-06-02,19.170720,28978,EA1,2020-04-01,0,73200.0,0.2652,2020-10-01,73200.0,0.050841
60,14676236,2020-06-03,23.555111,28978,EA1,2020-04-01,0,73200.0,0.2652,2020-10-01,73200.0,0.062468
90,14676236,2020-06-04,18.220712,28978,EA1,2020-04-01,0,73200.0,0.2652,2020-10-01,73200.0,0.048321
120,14676236,2020-06-05,14.196134,28978,EA1,2020-04-01,0,73200.0,0.2652,2020-10-01,73200.0,0.037648
...,...,...,...,...,...,...,...,...,...,...,...,...
102223,88357331,2022-09-27,441.014725,484399,SE2,2022-04-01,73200,732000.0,0.3647,2022-10-01,732000.0,1.608381
102253,88357331,2022-09-28,441.512055,484399,SE2,2022-04-01,73200,732000.0,0.3647,2022-10-01,732000.0,1.610194
102283,88357331,2022-09-29,437.240657,484399,SE2,2022-04-01,73200,732000.0,0.3647,2022-10-01,732000.0,1.594617
102313,88357331,2022-09-30,517.608354,484399,SE2,2022-04-01,73200,732000.0,0.3647,2022-10-01,732000.0,1.887718


In [80]:
# Result table exercise 1
df.groupby('meter_id')[['kwh', 'cost_pound']].sum().round(2).rename(columns={'meter_id': 'Meter ID', 'kwh': 'Total Estimated Consumption (kWh)', 'cost_pound': 'Total Cost (£)'})

,Total Estimated Consumption (kWh),Total Cost (£)
meter_id,,
14676236,28978.0,100.15
34509937,78324.0,275.49
50264822,265667.0,731.24
88357331,484399.0,1433.16


## Exercise 2: Random meter generator

In [96]:
def random_meter_list(n: int) -> pd.DataFrame:
    return pd.DataFrame(np.array([np.random.randint(1e8, 1e9, size=n),
                         np.random.randint(25e3, 5e5, size=n),
                         np.random.choice(df_rate_table.exit_zone.unique(), size=n)]).T, columns=df_meter_list.columns)

In [98]:
random_meter_list(5)

,meter_id,aq_kwh,exit_zone
0,270874881,125229,LC
1,750062885,445097,NO2
2,831866923,227405,NO1
3,623055754,440923,WA1
4,403701922,415986,EA2


## Exercise 3: Mock consumption data generator

In [116]:
def mock_consumption_data(meter_list: pd.DataFrame, start_date: datetime, duration_in_days: int) -> pd.DataFrame:
    date = pd.date_range(start_date, start_date + pd.offsets.DateOffset(days=duration_in_days), inclusive='left')
    mock_data = []
    for meter in meter_list['meter_id']:
        mock_data.append(pd.DataFrame({'meter_id': [meter]*duration_in_days,
                                       'date': date,
                                       'kwh': np.random.rand(duration_in_days) * 1100 + 2.5}))
    return pd.concat(mock_data)

In [122]:
mock_consumption_data(random_meter_list(5), start_date=datetime(2020,1,1), duration_in_days=5)

,meter_id,date,kwh
0,464226811,2020-01-01,964.529330
1,464226811,2020-01-02,307.737341
2,464226811,2020-01-03,956.953990
3,464226811,2020-01-04,133.196963
4,464226811,2020-01-05,1083.531727
0,160012012,2020-01-01,686.177189
1,160012012,2020-01-02,719.206603
2,160012012,2020-01-03,211.526593
3,160012012,2020-01-04,743.593638
4,160012012,2020-01-05,445.958985
